## 1. Check GPU and Setup Environment

In [ ]:
import torch

print("="*70)
print("🔍 Checking GPU Availability")
print("="*70)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️  WARNING: No GPU detected! Training will be very slow.")
    print("Go to Runtime > Change runtime type > Select GPU")

## 2. Clone Repository and Navigate to Project

In [ ]:
# Clone your repository
!git clone https://github.com/Zinoug/codeLLM-python-finetuning.git
%cd codeLLM-python-finetuning

# List files to confirm
!ls -lh final_data/

## 3. Install Dependencies

In [ ]:
# Install required packages
!pip install -q transformers datasets evaluate rouge-score sacrebleu tqdm

print("✅ Dependencies installed successfully!")

## 4. Verify Dataset

In [ ]:
import json
from collections import Counter

print("📊 Dataset Summary:\n")

for split in ['train', 'val', 'test']:
    filepath = f'final_data/{split}.jsonl'
    tasks = []
    
    with open(filepath, 'r') as f:
        for line in f:
            sample = json.loads(line)
            tasks.append(sample['task'])
    
    counts = Counter(tasks)
    print(f"{split.upper()}.jsonl: {sum(counts.values())} samples")
    for task, count in sorted(counts.items()):
        print(f"  • {task}: {count}")
    print()

## 5. Fine-tune Model

**Note:** Training will take approximately 2-3 hours on a T4 GPU.

If you get **Out of Memory** errors:
- Reduce `per_device_train_batch_size` to 1
- Increase `gradient_accumulation_steps` to 8

In [ ]:
# Run the fine-tuning script
!python finetune_codet5_multitask.py

## 6. Download Trained Model

After training completes, download your model to use locally.

In [ ]:
# Compress the final model
!tar -czf codet5_multitask_final.tar.gz codet5_multitask_final/

print("✅ Model compressed!")
print("\n📥 Download the model:")

from google.colab import files
files.download('codet5_multitask_final.tar.gz')

print("\n💡 To use locally, extract with: tar -xzf codet5_multitask_final.tar.gz")

## 7. (Optional) Quick Evaluation

Test a few samples to verify the model works.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

print("Loading model for quick test...")
tokenizer = AutoTokenizer.from_pretrained("codet5_multitask_final")
model = AutoModelForSeq2SeqLM.from_pretrained("codet5_multitask_final")

if torch.cuda.is_available():
    model = model.cuda()

def predict(text, max_length=128):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=max_length, num_beams=4)
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test examples
print("\n" + "="*70)
print("🧪 Quick Tests")
print("="*70)

# Test 1: Code Repair
buggy = """fix bug:
def add(a, b):
    return a - b"""
print("\n1. Code Repair:")
print(f"Input: {buggy}")
print(f"Output: {predict(buggy)}")

# Test 2: Bug Detection
code = """classify code:
def add(a, b):
    return a + b"""
print("\n2. Bug Detection:")
print(f"Input: {code}")
print(f"Output: {predict(code, max_length=10)}")

# Test 3: Code Summary
func = """summarize code:
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)"""
print("\n3. Code Summary:")
print(f"Input: {func}")
print(f"Output: {predict(func, max_length=64)}")

print("\n" + "="*70)
print("✨ Model is working!")
print("="*70)

## 8. (Optional) Mount Google Drive

Save model directly to Google Drive to avoid re-downloading.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Copy model to Drive
!cp -r codet5_multitask_final /content/drive/MyDrive/

print("✅ Model saved to Google Drive: MyDrive/codet5_multitask_final/")

---

## 📋 Troubleshooting

### Out of Memory Error
Edit `finetune_codet5_multitask.py` and reduce batch size:
```python
per_device_train_batch_size=1,  # Reduce from 2 to 1
gradient_accumulation_steps=8,  # Increase from 4 to 8
```

### Session Timeout
Colab disconnects after 12 hours. To continue training:
1. The script saves checkpoints every 500 steps
2. Edit `finetune_codet5_multitask.py` to resume from checkpoint:
```python
trainer.train(resume_from_checkpoint="codet5_multitask_checkpoint/checkpoint-500")
```

### Slow Training
- Make sure GPU is enabled (Runtime > Change runtime type)
- T4 GPU: ~2-3 hours
- CPU: ~20+ hours (not recommended)

---

## 🎯 Next Steps

After training:
1. Download model with cell #6
2. Run evaluation locally: `python evaluate_model.py`
3. Check pass@1 score for code repair task

---

**Academic Project** - Seoul National University of Technology and Science